In [ ]:
import numpy as np
import traci
import os
import sys
import sumolib
from sumolib.net import readNet
from utility import getEdgeIndex
from utility import valid_neighbors

In [ ]:
# SALERNO NET ------------
net_name = r"salerno_net/osmcars.net.xml"
sumocfg_name = r"salerno_net/osm.copy.sumocfg"
behaviors_name = r"salerno_net/salerno_behaviors.npy"
#targets are parking spaces
targets = ['672732730', '-672732731','-672732728', '401420254#0','-565381618#0', '401420254#4', '-1001198066',
          '744932253', '670934106', '670934108', '160821659#2', '671983008', '92961457#3', '92961457#4', 
          '92961462', '672273418#3', '673737658#3', '673737658#6', '-565381618#0', '163563222', 'E5', 'E6'] 


In [ ]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

#Choose between these lines for GUI or not
#sumoBinary = sumolib.checkBinary('sumo-gui')
sumoBinary = sumolib.checkBinary('sumo')

sumoCmd = [sumoBinary, "-c", sumocfg_name,"--step-length","0.05"] #The last parameter is the step size, has to be small
traci.start(sumoCmd)
print("Starting SUMO...")
    
net = readNet(net_name) #read road network
edge_list = net.getEdges()

#Dictionary of list IDs
edge_dict = {}

for i in range(len(edge_list)):
    edge_dict[i] = edge_list[i].getID()

behaviors = np.zeros((len(targets), len(edge_list), len(edge_list)))

i=0
for t in targets:
    pmfs = np.zeros((len(edge_list), len(edge_list)))
    for e in edge_list: 
        
        route = traci.simulation.findRoute(e.getID(), t).edges
        
        pmf = [0]*((len(edge_list)))
        
        if len(route) ==0 : # edges are not connected 
            for x in range(len(pmf)):
                pmf[x] = 0
        
        elif(len(route) == 1):
            e_prime = e.getID()
            
            pmf[getEdgeIndex(e_prime, edge_dict)] = 0.95  
            valid_edges = valid_neighbors(getEdgeIndex(e.getID(), edge_dict), edge_dict, net)
                
            for v in valid_edges:
                if(v!= getEdgeIndex(e_prime, edge_dict)):
                    pmf[v] = (1-prob) / len(valid_edges)
        else:
               
            e_prime = route[1] # e' is the edge just after e 

            prob = 0.95 
            pmf[getEdgeIndex(e_prime, edge_dict)] = prob

            valid_edges = valid_neighbors(getEdgeIndex(e.getID(), edge_dict), edge_dict, net)

            for v in valid_edges:
                if(v!= getEdgeIndex(e_prime, edge_dict)):
                    pmf[v] = (1-prob) / len(valid_edges)
            

        pmf = np.array(pmf)
                
        if np.sum(pmf) !=0:
            pmf = pmf/np.sum(pmf)
                
        pmfs[getEdgeIndex(e.getID(), edge_dict)] = pmf
        
    behaviors[i] = pmfs


    i+=1
print("Behaviors generated!")


In [ ]:
traci.close()

In [ ]:
np.save(behaviors_name, behaviors)